In [ ]:
import os
from IPython.display import display
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
from onekey_algo import OnekeyDS as okds
from onekey_algo import get_param_in_cwd

os.makedirs('img', exist_ok=True)
os.makedirs('results', exist_ok=True)
os.makedirs('features', exist_ok=True)


task_type = 'metastasis'

group_info = get_param_in_cwd('dataset_column') or 'group'
labelf = r'E:/20230904-LiuHui/metastasis.csv'

labels = [get_param_in_cwd('task_column') or 'label']

In [ ]:
from collections import namedtuple
import onekey_algo.custom.components as okcomp
from onekey_algo import OnekeyDS as okds
from onekey_algo.custom.utils import print_join_info
from onekey_algo import get_param_in_cwd

import pandas as pd
import numpy as np
import os
os.makedirs('img', exist_ok=True)
os.makedirs('features', exist_ok=True)

def mapID(x):
    items = x[:-2].split('-')
    return f"20{items[0]}-{int(items[1][:5]):05d}"

prob_histo = pd.read_csv('features/path_prob_histogram.csv')
prob_tfidf = pd.read_csv('features/path_prob_tfidf.csv')
prob = pd.merge(prob_histo, prob_tfidf, on='ID', how='inner', suffixes=['_histo', '_tfidf'])
prob['ID'] = prob['ID'].astype(str)

pred_histo = pd.read_csv('features/path_pred_histogram.csv')
pred_tfidf = pd.read_csv('features/path_pred_tfidf.csv')
pred = pd.merge(pred_histo, pred_tfidf, on='ID', how='inner', suffixes=['_histo', '_tfidf'])
pred['ID'] = pred['ID'].astype(str)

features = pd.merge(prob, pred, on='ID', how='inner')
features['ID'] = features['ID'].map(mapID)
features.to_csv('features/path_features.csv', index=False, header=True)
labels = ['label']
featrues_not_use = ['ID']

label_data = pd.read_csv(labelf, header=0)
label_data['ID'] = label_data['ID'].map(lambda x: x.replace('.nii.gz', ''))
combined_data = pd.merge(features, label_data, left_on='ID', right_on='ID', how='inner')
combined_data = pd.merge(combined_data, pd.read_csv(get_param_in_cwd('label_file'))[['ID', 'group']], on='ID', how='inner')
combined_data

In [ ]:
from onekey_algo.custom.utils import print_join_info

ids = combined_data['ID']
combined_data = combined_data.drop(['ID'], axis=1)
print(combined_data[labels].value_counts())
combined_data

In [ ]:
combined_data.describe()

In [ ]:
from onekey_algo.custom.components.comp1 import normalize_df
# data = normalize_df(combined_data, not_norm=labels, group=group_info)
# data = data.dropna(axis=1)
data = combined_data
data.describe()

In [ ]:
import seaborn as sns
from onekey_algo.custom.components.stats import clinic_stats

stats = clinic_stats(data[data[group_info] == 'train'], stats_columns=list(data.columns[0:-2]), label_column=labels[0], 
                     continuous_columns=list(data.columns[0:-2]))
stats

In [ ]:
import matplotlib.pyplot as plt

def map2float(x):
    try:
        return float(str(x)[1:])
    except:
        return 1

stats[['pvalue']] = stats[['pvalue']].applymap(map2float)
stats[['group']] = stats[['feature_name']].applymap(lambda x: 'BoW' if '-' in x else 'PLH')
stats = stats[['feature_name', 'pvalue', 'group']]
g = sns.catplot(x="group", y="pvalue", data=stats, kind="violin")
g.fig.set_size_inches(15,10)
sns.stripplot(x="group", y="pvalue", data=stats, ax=g.ax, color='black')
plt.savefig(f'img/{task_type}_feature_stats.svg', bbox_inches = 'tight')

In [ ]:
pvalue = 0.05
sel_feature = list(stats[stats['pvalue'] < pvalue]['feature_name']) + labels + [group_info]
data = data#[sel_feature]
data

In [ ]:
pearson_corr = data[data[group_info] == 'train'][[c for c in data.columns if c not in labels]].corr('pearson')
# kendall_corr = data[[c for c in data.columns if c not in labels]].corr('kendall')
# spearman_corr = data[[c for c in data.columns if c not in labels]].corr('spearman')

In [ ]:
import seaborn as sns
from onekey_algo.custom.components.comp1 import draw_matrix
# plt.figure(figsize=(50.0, 40.0))

# draw_matrix(pearson_corr, annot=True, cmap='YlGnBu', cbar=False)
# plt.savefig(f'img/Rad_feature_corr.svg', bbox_inches = 'tight')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# pp = sns.clustermap(pearson_corr, linewidths=.5, figsize=(50.0, 40.0), cmap='YlGnBu')
# plt.setp(pp.ax_heatmap.get_yticklabels(), rotation=0)
# plt.savefig(f'img/Rad_feature_cluster.svg', bbox_inches = 'tight')

In [ ]:
from onekey_algo.custom.components.comp1 import select_feature, select_feature_mrmr
sel_feature = select_feature(pearson_corr, threshold=0.9, topn=10, verbose=False)
# sel_feature = select_feature_mrmr(data[sel_feature + labels], num_features=16)
fids = features['ID']
features = features[sel_feature]
sel_feature = sel_feature + labels + [group_info]
sel_feature

In [ ]:
sel_data = data[sel_feature]
sel_data

In [ ]:
from onekey_algo.custom.components.comp1 import analysis_features
analysis_features(data[sel_feature[:-2]], data[labels[0]], prefix='rad_mil_', save_dir='img')

In [ ]:
import numpy as np
import onekey_algo.custom.components as okcomp

n_classes = 2
train_data = sel_data[(sel_data[group_info] == 'train')]
train_ids = ids[train_data.index]
train_data = train_data.reset_index()
train_data = train_data.drop('index', axis=1)
y_data = train_data[labels]
X_data = train_data.drop(labels + [group_info], axis=1)

val_data = sel_data[sel_data[group_info] != 'val']
val_ids = ids[val_data.index]
val_data = val_data.reset_index()
val_data = val_data.drop('index', axis=1)
y_val_data = val_data[labels]
X_val_data = val_data.drop(labels + [group_info], axis=1)

test_data = sel_data[sel_data[group_info] == 'test']
test_ids = ids[test_data.index]
test_data = test_data.reset_index()
test_data = test_data.drop('index', axis=1)
y_test_data = test_data[labels]
X_test_data = test_data.drop(labels + [group_info], axis=1)

y_all_data = sel_data[labels]
X_all_data = sel_data.drop(labels + [group_info], axis=1)

column_names = X_data.columns

In [ ]:
y_test_data['label'].value_counts()

In [ ]:
alpha = okcomp.comp1.lasso_cv_coefs(X_data, y_data, column_names=None)
plt.savefig(f'img/{task_type}_feature_lasso.svg', bbox_inches = 'tight')

In [ ]:
okcomp.comp1.lasso_cv_efficiency(X_data, y_data, points=50)
plt.savefig(f'img/{task_type}_feature_mse.svg', bbox_inches = 'tight')

In [ ]:
from sklearn import linear_model

models = []
for label in labels:
    clf = linear_model.Lasso(alpha=alpha)
    clf.fit(X_data, y_data[label])
    models.append(clf)

In [ ]:
model_names = get_param_in_cwd('ml_models')
models = okcomp.comp1.create_clf_model(model_names)
model_names = list(models.keys())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score

results = okcomp.comp1.get_bst_split(X_data, y_data, models, test_size=0.3, metric_fn=roc_auc_score, n_trails=4, cv=True, random_state=0)
_, (X_train_sel, X_test_sel, y_train_sel, y_test_sel) = results['results'][2]
X_train_sel, X_val_sel, X_test_sel, y_train_sel, y_val_sel, y_test_sel = X_data, X_val_data, X_test_data, y_data, y_val_data, y_test_data
trails, _ = zip(*results['results'])
cv_results = pd.DataFrame(trails, columns=model_names)
sns.barplot(data=cv_results)
plt.ylabel('AUC %')
plt.xlabel('Model Nmae')
plt.xticks(rotation=90)
plt.ylim(0.5,)
plt.savefig(f'img/{task_type}Rad_model_cv.svg', bbox_inches = 'tight')

In [ ]:
import joblib
from onekey_algo.custom.components.comp1 import plot_feature_importance
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from onekey_algo.custom.components.comp1 import smote_resample

targets = []
os.makedirs('models', exist_ok=True)
for l in labels:
    new_models = okcomp.comp1.create_clf_model_none_overfit(model_names)
#     new_models['LR'] = LogisticRegression(penalty='l2')
#     new_models['SVM'] = SVC(kernel='rbf', probability=True)
    new_models['RandomForest'] = RandomForestClassifier(n_estimators=14, max_depth=2,
                                                        min_samples_split=3, random_state=0)
    new_models['XGBoost'] = XGBClassifier(n_estimators=3, objective='binary:logistic', max_depth=3, min_child_weight=.35,
                                          use_label_encoder=False, eval_metric='error')
    new_models['LightGBM'] = LGBMClassifier(n_estimators=12,  max_depth=2,)
#     new_models['ExtraTrees'] = ExtraTreesClassifier(n_estimators=4, max_depth=4, min_samples_split=3, random_state=0)
#     new_models['MLP'] = MLPClassifier(hidden_layer_sizes=(64, 32, 16), max_iter=100, solver='sgd', random_state=0)
    model_names = list(new_models.keys())
    new_models = list(new_models.values())
    for mn, m in zip(model_names, new_models):
        X_data_smote, y_data_smote = X_data, y_data[l]
#         X_data_smote, y_data_smote = smote_resample(X_data, y_data[l])
        m.fit(X_data_smote, y_data_smote)
    targets.append(new_models)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from onekey_algo.custom.components.delong import calc_95_CI
from onekey_algo.custom.components.metrics import analysis_pred_binary

metric = []
pred_sel_idx = []
predictions = [[(model.predict(X_train_sel), model.predict(X_test_sel))  
                for model in target] for label, target in zip(labels, targets)]
pred_scores = [[(model.predict_proba(X_train_sel), model.predict_proba(X_test_sel)) 
                for model in target] for label, target in zip(labels, targets)]
for label, prediction, scores in zip(labels, predictions, pred_scores):
    pred_sel_idx_label = []
    for mname, (train_pred, test_pred), (train_score, test_score) in zip(model_names, prediction, scores):
        acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y_train_sel[label], 
                                                                                              train_score[:, 1], use_youden=False)
        ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
        metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"Train"))

        acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y_test_sel[label], 
                                                                                              test_score[:, 1], use_youden=False)
        ci = f"{ci[0]:.4f} - {ci[1]:.4f}"
        metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, 'Test'))
        pred_sel_idx_label.append(np.logical_or(test_score[:, 0] >= thres, test_score[:, 1] >= thres))

    pred_sel_idx.append(pred_sel_idx_label)
metric = pd.DataFrame(metric, index=None, columns=['model_name', 'Accuracy', 'AUC', '95% CI',
                                                   'Sensitivity', 'Specificity', 
                                                   'PPV', 'NPV', 'Precision', 'Recall', 'F1',
                                                   'Threshold', 'Task'])
metric[metric['Task'] != 'Val']

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 10))
plt.subplot(211)
sns.barplot(x='model_name', y='Accuracy', data=metric, hue='Task')
plt.subplot(212)
sns.lineplot(x='model_name', y='Accuracy', data=metric, hue='Task')
plt.savefig(f'img/{task_type}_model_acc.svg', bbox_inches = 'tight')

In [ ]:
sel_model = model_names

for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
    
        # Plot all ROC curves
        plt.figure(figsize=(8, 8))
        for pred_score, label in zip(pred_scores, labels):
            okcomp.comp1.draw_roc([np.array(y_train_sel[label]), np.array(y_test_sel[label])], 
                                  pred_score[sel_model_idx], 
                                  labels=['Train', 'Test'], title=f"Model: {sm}")
            plt.savefig(f'img/{task_type}_model_{sm}_roc.svg', bbox_inches = 'tight')
            plt.show()

In [ ]:
sel_model = model_names

for pred_score, label in zip(pred_scores, labels):
    pred_val_scores = []
    pred_test_scores = []
    
    for sm in sel_model:
        if sm in model_names:
            sel_model_idx = model_names.index(sm)
            pred_val_scores.append(pred_score[sel_model_idx][0])
            pred_test_scores.append(pred_score[sel_model_idx][1])
    okcomp.comp1.draw_roc([np.array(y_train_sel[label])] * len(pred_val_scores), 
                          pred_val_scores, 
                          labels=sel_model, title=f"Model AUC")
    plt.savefig(f'img/{task_type}_model_train_roc.svg', bbox_inches = 'tight')
    plt.show()
    okcomp.comp1.draw_roc([np.array(y_test_sel[label])] * len(pred_test_scores), 
                          pred_test_scores, 
                          labels=sel_model, title=f"Model AUC")
    plt.savefig(f'img/{task_type}_model_test_roc.svg', bbox_inches = 'tight')
    plt.show()

In [ ]:
from onekey_algo.custom.components.comp1 import plot_DCA

for pred_score, label in zip(pred_scores, labels):
    pred_test_scores = []
    for sm in sel_model:
        if sm in model_names:
            sel_model_idx = model_names.index(sm)
            okcomp.comp1.plot_DCA(pred_score[sel_model_idx][0][:,1], np.array(y_train_sel[label]),
                                  title=f'Model {sm} DCA')
            plt.savefig(f'img/{task_type}_model_train_{sm}_dca.svg', bbox_inches = 'tight')
            plt.show()
            okcomp.comp1.plot_DCA(pred_score[sel_model_idx][1][:,1], np.array(y_test_sel[label]),
                                  title=f'Model {sm} DCA')
            plt.savefig(f'img/{task_type}_model_test_{sm}_dca.svg', bbox_inches = 'tight')

In [ ]:

sel_model = model_names
c_matrix = {}

for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
        for idx, label in enumerate(labels):
            cm = okcomp.comp1.calc_confusion_matrix(predictions[idx][sel_model_idx][0], y_train_sel[label],
#                                                     sel_idx = pred_sel_idx[idx][sel_model_idx],
                                                    class_mapping={1:'1', 0:'0'}, num_classes=2)
            c_matrix[label] = cm
            plt.figure(figsize=(5, 4))
            plt.title(f'Model:{sm}')
            okcomp.comp1.draw_matrix(cm, norm=False, annot=True, cmap='Blues', fmt='.0f')
            plt.savefig(f'img/{task_type}_model_train_{sm}_cm.svg', bbox_inches = 'tight')
            

for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
        for idx, label in enumerate(labels):
            cm = okcomp.comp1.calc_confusion_matrix(predictions[idx][sel_model_idx][1], y_test_sel[label],
#                                                     sel_idx = pred_sel_idx[idx][sel_model_idx],
                                                    class_mapping={1:'1', 0:'0'}, num_classes=2)
            c_matrix[label] = cm
            plt.figure(figsize=(5, 4))
            plt.title(f'Model:{sm}')
            okcomp.comp1.draw_matrix(cm, norm=False, annot=True, cmap='Blues', fmt='.0f')
            plt.savefig(f'img/{task_type}_model_test_{sm}_cm.svg', bbox_inches = 'tight')

In [ ]:
sel_model = model_names
c_matrix = {}

for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
        for idx, label in enumerate(labels):            
            okcomp.comp1.draw_predict_score(pred_scores[idx][sel_model_idx][0], y_train_sel[label])
            plt.title(f'{sm} sample predict score')
            plt.legend(labels=["label=0", "label=1"],loc="lower right") 
            plt.savefig(f'img/{task_type}_train_{sm}_sample_dis.svg', bbox_inches = 'tight')
            plt.show()
  
            okcomp.comp1.draw_predict_score(pred_scores[idx][sel_model_idx][1], y_test_sel[label])
            plt.title(f'{sm} sample predict score')
            plt.legend(labels=["label=0", "label=1"],loc="lower right") 
            plt.savefig(f'img/{task_type}_test_{sm}_sample_dis.svg', bbox_inches = 'tight')
            plt.show()

In [ ]:
import os
import numpy as np

os.makedirs('results', exist_ok=True)
sel_model = sel_model

for idx, label in enumerate(labels):
    for sm in sel_model:
        if sm in model_names:
            sel_model_idx = model_names.index(sm)
            target = targets[idx][sel_model_idx]
            train_indexes = np.reshape(np.array(train_ids), (-1, 1)).astype(str)
            val_indexes = np.reshape(np.array(fids), (-1, 1)).astype(str)
            test_indexes = np.reshape(np.array(test_ids), (-1, 1)).astype(str)
            y_train_pred_scores = target.predict_proba(X_train_sel)
            y_val_pred_scores = target.predict_proba(features)
            y_test_pred_scores = target.predict_proba(X_test_sel)
            columns = ['ID'] + [f"{label}-{i}"for i in range(y_test_pred_scores.shape[1])]
            result_train = pd.DataFrame(np.concatenate([train_indexes, y_train_pred_scores], axis=1), columns=columns)
#             result_train['ID'] = result_train['ID'].map(lambda x: f"{x}.nii.gz")
            result_train.to_csv(f'results/{task_type}_{sm}_train.csv', index=False)
            result_val = pd.DataFrame(np.concatenate([val_indexes, y_val_pred_scores], axis=1), columns=columns)
            result_val.to_csv(f'results/{task_type}_{sm}_all.csv', index=False)
            result_test = pd.DataFrame(np.concatenate([test_indexes, y_test_pred_scores], axis=1), columns=columns)
#             result_test['ID'] = result_test['ID'].map(lambda x: f"{x}.nii.gz")
            result_test.to_csv(f'results/{task_type}_{sm}_test.csv', index=False)